### 0.데이터 다운로드


In [ ]:
!git clone https://github.com/Soyoung-Yoon/data_02

Cloning into 'data_02'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 72 (delta 7), reused 0 (delta 0), pack-reused 47
Unpacking objects: 100% (72/72), 448.80 KiB | 2.09 MiB/s, done.


In [ ]:
import scipy
scipy.__version__

In [ ]:
!pip install scipy==1.6.0

### 1.카이제곱 적합도 검정

####1-1.특정비율
A학급 40명의 혈액형 비율을 A, B, O, AB 각각 20%, 20%, 30%, 30%로 예상하였다. 실제 측정 결과 16, 16, 4, 4 명인 경우의 적합도 검정을 수행하여 보자


- 𝑯_𝟎 : 변수의 분포가 기대 분포와 같다
- 𝑯_𝟏 : 변수의 분포가 기대 분포와 같지 않다


In [ ]:
# [파일로 주어진 경우 직접 개수 계산]
import pandas as pd
data = {'blood_type': ['A']*16 + ['B']*16 + ['O']*4 + ['AB']*4}
data = pd.DataFrame(data)
data['blood_type'] = pd.Categorical(data['blood_type'], 
                                    categories=['A', 'B', 'O', 'AB'])
#print(data)
observed = data['blood_type'].value_counts().sort_index()
print(observed.to_list())

[16, 16, 4, 4]


In [ ]:
from scipy.stats import chisquare
import numpy as np
observed = [16, 16, 4, 4]
expected = sum(observed) * np.array([0.2, 0.2, 0.3, 0.3])  #[8, 8, 12, 12]
print(expected)
statistic, pvalue = chisquare(observed, expected)
print(f'chisq={statistic:.2f}, p-value={pvalue:.2f}')
print('기각' if pvalue < 0.05 else '채택')
print('변수의 분포가 기대 분포와 같지 않다')

[ 8.  8. 12. 12.]
chisq=26.67, p-value=0.00
기각
변수의 분포가 기대 분포와 같지 않다


#### 1-2.동일비율
4개의 범주에 대해 동일한 비율이라고 가정하고, 실제측정 한 표본 분포가 다음과 같을 때, 카이제곱 적합도 검정을 수행하여 보자

         observed = [54, 46, 60, 40]


- 𝑯_𝟎 : 변수의 분포에 비율 차이가 없다.
- 𝑯_𝟏 : 변수의 분포에 비율 차이가 있다.

In [ ]:
from scipy.stats import chisquare
import numpy as np
observed = [54, 46, 60, 40]
expected = sum(observed) * np.array([0.25]*4)
print(expected)
statistic, pvalue = chisquare(observed, expected)  # OE
print(f'chisq={statistic:.2f}, p-value={pvalue:.2f}')
print('기각' if pvalue < 0.05 else '채택')
print('변수의 분포에 비율 차이가 없다')

[50. 50. 50. 50.]
chisq=4.64, p-value=0.20
채택
변수의 분포에 비율 차이가 없다


### 2.카이제곱 동질성/독립성 검정
- 두 범주형 자료 간의 차이 및 연관성을 분석하는 검증 방법
- 관찰빈도(실제값)가 기대빈도(기댓값)와 유의한 차이가 있는지 검증
- 귀무가설이 기각되면 범주별 기대빈도와 관측빈도의 차이가 크다/독립이 아니다라고 할 수 있음
- 기대빈도 5미만의 셀이 전체의 20% 보다 작아야 함 --> 데이터 양이 적은 경우 Fisher’s Exact Test 사용 (2 * 2)


#### 2-1.카이제곱 동질성 검정



다음의 교차표를 사용하여 성별별 핸드폰 모델 선호도 분포가 같은지 검정하라.

           model_A  model_B  model_C
        M       10       40       50
        F       30       60       10

- 핸드폰 모델만 변수로 보고 성별은 비교 대상이 되는 그룹으로 해석
- 남자 그룹, 여자그룹의 표본을 각각 추출하여 각 표본의 핸드폰 모델 선호도를 조사함
- H0 : 성별별 핸드폰 모델 선호도 분포는 같다
- H1 : 성별별 핸드폰 모델 선호도 분포가 같지 않다        

In [ ]:
from scipy.stats import chi2_contingency
print(help(chi2_contingency))

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

crs_table = pd.DataFrame([[10, 40, 50], [30, 60, 10]], 
                         index=['M', 'F'], 
                columns=['model_A', 'model_B', 'model_C'])

print(crs_table)
chi_statistic, p, df, expected = chi2_contingency(crs_table)
print(f'chi 스퀘어 값: {chi_statistic:.4f}',
      f'p-value (0.05): {p:.4f}',
      f'자유도 수: {df}',
      f'기대값:\n{pd.DataFrame(expected)}',
      f'측정값:\n{crs_table}', sep='\n')
print('기각' if p < 0.05 else '채택')
# 대립가설 : 성별별 핸드폰 모델 선호도 분포가 같지 않다

   model_A  model_B  model_C
M       10       40       50
F       30       60       10
chi 스퀘어 값: 40.6667
p-value (0.05): 0.0000
자유도 수: 2
기대값:
      0     1     2
0  20.0  50.0  30.0
1  20.0  50.0  30.0
측정값:
   model_A  model_B  model_C
M       10       40       50
F       30       60       10
기각


####2-2.카이제곱 독립성 검정
data_chi.csv 파일을 사용하여 당뇨와 비만 사이의 관계가 독립인지 검정하라.
- 귀무가설 : 당뇨와 비만 사이에 관계는 독립이다.
-대립가설 : 당뇨와 비만 사이에 관계는 독립이 아니다.

In [ ]:
# [1] 데이터 가져오기
import pandas as pd
from scipy.stats import chi2_contingency
data = pd.read_csv('./data_02/data_chi.csv')
print(data)

   당뇨 여부 비만 여부
0      Y     N
1      N     Y
2      N     N
3      N     N
4      N     Y
..   ...   ...
95     N     N
96     N     N
97     Y     N
98     N     N
99     N     N

[100 rows x 2 columns]


In [ ]:
# [2] 카이제곱 분석을 위한 분할표 만들기
crs_table = pd.crosstab(data['당뇨 여부'], data['비만 여부'])
print(crs_table)

비만 여부   N   Y
당뇨 여부        
N      62  22
Y      12   4


In [ ]:
# [3] 카이스퀘어 분석
chi_statistic, p, df, expected = chi2_contingency(crs_table)
print(f'chi 스퀘어 값: {chi_statistic:.4f}',
      f'p-value (0.05): {p:.4f}',
      f'자유도 수: {df}',
      f'기대값:\n{pd.DataFrame(expected)}',
      f'측정값:\n{crs_table}', sep='\n')
print('기각' if p < 0.05 else '채택')

chi 스퀘어 값: 0.0000
p-value (0.05): 1.0000
자유도 수: 1
기대값:
       0      1
0  62.16  21.84
1  11.84   4.16
측정값:
비만 여부   N   Y
당뇨 여부        
N      62  22
Y      12   4
채택


In [ ]:
# [4] 결론
# p > 0.05 이므로, 귀무가설을 채택
# 당뇨와 비만 사이에 관계는 독립이다(연관성이 없다)

#### 2-3.피셔의 정확검정
- 만약 사건 발생 수, 빈도 수가 극히 적거나(특히 5개 미만의 빈도가 전체 셀 중 20%이상 존재하는 경우) 서로의 빈도 수의 차이가 많이나는 경우에는 카이제곱 검정의 정확도는 낮아집니다. 
- 이때 필요한 검정이 바로 '피셔의 정확검정'입니다.

가설설정
- 귀무가설 : 약과 효과의 관계는 독립이다. 
- 대립가설 : 약과 효과의 관계가 독립이 아니다.

In [ ]:
import pandas as pd
from scipy.stats import fisher_exact

# [1] 분할표 생성
data = pd.DataFrame([[1, 6], [5, 2]])
data.columns = ['가짜 약','진짜 약']
data.index = ['효과있음', '효과없음']
print(data)

      가짜 약  진짜 약
효과있음     1     6
효과없음     5     2


In [ ]:
# [2] 피셔의 정확 검정
oddsratio, pvalue = fisher_exact(data)
print(f'oddsratio:{oddsratio:.4f}, p-value: {pvalue:.4f}')

oddsratio:0.0667, p-value: 0.1026


In [ ]:
# [3] 결과
print('기각' if pvalue < 0.05 else '채택')
# pvalue >= 0.05 이기 때문에 귀무가설을 채택
# 약과 효과의 관계는 독립이다.(연관성이 없음)

채택


####2-4.아내와 남편의 집안일
- 귀무가설 : 집안일의 종류와 수행하는 사람의 관계는 독립이다.
- 대립가설 : 집안일의 종류와 수행하는 사람의 관계가 독립이 아니다.

In [ ]:
# [1] crosstable 형태의 데이터 읽기
import pandas as pd

data = pd.read_csv('data_02/housetasks.csv', index_col=0)
print(data.shape)
data.info()

(13, 4)
<class 'pandas.core.frame.DataFrame'>
Index: 13 entries, Laundry to Holidays
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Wife         13 non-null     int64
 1   Alternating  13 non-null     int64
 2   Husband      13 non-null     int64
 3   Jointly      13 non-null     int64
dtypes: int64(4)
memory usage: 520.0+ bytes


In [ ]:
print(data.head(5))

            Wife  Alternating  Husband  Jointly
Laundry      156           14        2        4
Main_meal    124           20        5        4
Dinner        77           11        7       13
Breakfeast    82           36       15        7
Tidying       53           11        1       57


In [ ]:
# [2] 카이제곱 독립성 검정
from scipy.stats import chi2_contingency

chi_statistic, p, df, expected = chi2_contingency(data)
print(f'chi 스퀘어 값: {chi_statistic:.4f}',
      f'p-value (0.05): {p:.4f}',
      f'자유도 수: {df}',
      f'기대값:\n{pd.DataFrame(expected)}', sep='\n')

chi 스퀘어 값: 1944.4562
p-value (0.05): 0.0000
자유도 수: 36
기대값:
            0          1          2          3
0   60.550459  25.633028  38.449541  51.366972
1   52.637615  22.283257  33.424885  44.654243
2   37.155963  15.729358  23.594037  31.520642
3   48.165138  20.389908  30.584862  40.860092
4   41.972477  17.768349  26.652523  35.606651
5   38.876147  16.457569  24.686353  32.979931
6   41.284404  17.477064  26.215596  35.022936
7   33.027523  13.981651  20.972477  28.018349
8   47.821101  20.244266  30.366399  40.568234
9   38.876147  16.457569  24.686353  32.979931
10  47.821101  20.244266  30.366399  40.568234
11  56.766055  24.030963  36.046445  48.156537
12  55.045872  23.302752  34.954128  46.697248


In [ ]:
# [3] 결과
print('기각' if p < 0.05 else '채택')
# p-value < 0.05 때문에, 귀무가설을 기각
# 대립가설 : 집안일의 종류와 수행하는 사람의 관계가 독립이 아니다. (연관성이 있음)

기각


####2-5.분류 모델에서 활용
- 2개 범주형 변수의 관계에 대해 검정한다

In [ ]:
# [1] 파일 불러오기
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/1st_Train.csv')
X = data.drop(columns=['Reached.on.Time_Y.N'])
Y = data[['ID', 'Reached.on.Time_Y.N']]

In [ ]:
#[2] X,Y가 분리되어 있는 경우 병합하기
XY = pd.merge(X, Y, on='ID')

#[3] 컬럼별 값의 가짓수 확인하기
print(XY.nunique())

ID                     10999
Warehouse_block            5
Mode_of_Shipment           3
Customer_care_calls        6
Customer_rating            5
Cost_of_the_Product      215
Prior_purchases            8
Product_importance         3
Gender                     2
Discount_offered          65
Weight_in_gms           4034
Reached.on.Time_Y.N        2
dtype: int64


In [ ]:
#[4] 범주형 변수 확인하기 
# (설명을 보고 확인하는 것이 정확함)
XY.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10999 entries, 0 to 10998
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   10999 non-null  int64 
 1   Warehouse_block      10999 non-null  object
 2   Mode_of_Shipment     10999 non-null  object
 3   Customer_care_calls  10999 non-null  int64 
 4   Customer_rating      10999 non-null  int64 
 5   Cost_of_the_Product  10999 non-null  int64 
 6   Prior_purchases      10999 non-null  int64 
 7   Product_importance   10999 non-null  object
 8   Gender               10999 non-null  object
 9   Discount_offered     10999 non-null  int64 
 10  Weight_in_gms        10999 non-null  int64 
 11  Reached.on.Time_Y.N  10999 non-null  int64 
dtypes: int64(8), object(4)
memory usage: 1.1+ MB


- (독립)범주형 변수 : 'Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender', 'Customer_rating', 'Customer_care_calls', 'Prior_purchases'
- (종속)범주형 변수 : 'Reached.on.Time_Y.N'
- 귀무가설 : 'Reached.on.Time_Y.N'과 feature는 독립이다. (연관성이 없다.)
- 대립가설 : 'Reached.on.Time_Y.N'과 feature는 독립이 아니다. (연관성이 있다.)


In [ ]:
data['Customer_care_calls'].value_counts()

In [ ]:
# [5] 카이제곱 독립성 검정
from scipy.stats import chi2_contingency
feature = 'Customer_rating'
crs_table = pd.crosstab(index=data['Reached.on.Time_Y.N'], 
                        columns=data[feature])
chi_statistic, p, df, expected = chi2_contingency(crs_table)
print(f'chi 스퀘어 값: {chi_statistic:.4f}',
      f'p-value (0.05): {p:.4f}',
      f'자유도 수: {df}',
      f'기대값:\n{pd.DataFrame(expected)}', sep='\n')

chi 스퀘어 값: 3.2000
p-value (0.05): 0.5249
자유도 수: 4
기대값:
             0            1            2            3           4
0   901.396491   873.164833   903.009728   882.844259   875.58469
1  1333.603509  1291.835167  1335.990272  1306.155741  1295.41531


In [ ]:
# [6] 결론
print('기각' if p < 0.05 else '채택')
# p-value >= 0.05, 귀무가설을 채택
# 'Reached.on.Time_Y.N'과 feature는 독립이다.(연관성이 없음)

채택


In [ ]:
features = ['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 
            'Gender', 'Customer_rating', 'Customer_care_calls',
            'Prior_purchases']
target = 'Reached.on.Time_Y.N'

def my_chi2(data, target, feature):
    crs_table = pd.crosstab(data[target], data[feature])
    chi_statistic, p, df, expected = chi2_contingency(crs_table)
    # print(f'chi 스퀘어 값: {chi_statistic:.4f}',
    #     f'p-value (0.05): {p:.4f}',
    #     f'자유도 수: {df}',
    #     f'기대값:\n{pd.DataFrame(expected)}', sep='\n')
    return f"{feature}: {'기각' if p < 0.05 else '채택'}"

for x in features:
    result = my_chi2(data, 'Reached.on.Time_Y.N', x)
    print(result)

Warehouse_block: 채택
Mode_of_Shipment: 채택
Product_importance: 기각
Gender: 채택
Customer_rating: 채택
Customer_care_calls: 기각
Prior_purchases: 기각


### 3.Wilcoxon Signed Rank Test
- 일표본/대응표본 T검정에서 정규성 가정이 만족되지 않을 때 사용하는 비모수 검정법
- 일표본/대응표본 T검정과는 달리 **중앙값에 관한 결과를 얻을 수 있음**.
- 가정
  1. 표본은 동일한 모집단에서 추출되어야 한다.
  2. 표본은 임의, 독립적으로 추출되어야 한다.

#### 3-1.일표본(One Sample)
- alternative='two-sided'
   - H0: 모집단의 중앙값은 A 이다.
   - H1: 모집단의 중앙값은 A가 아니다.
   - H0: 값 차이(관찰값 - A)의 중앙값은 0이다(=값의 차이가 없다)
   - H1: 값 차이(관찰값 - A)의 중앙값은 0이 아니다(=값의 차이가 있다)   
- alternative='less'
   - H0: 모집단의 중앙값은 A보다 크거나 같다.
   - H1: 모집단의 중앙값은 A보다 작다.
   - H0: 값 차이(관찰값 - A)의 중앙값은 0보다 크거나 같다
   - H1: 값 차이(관찰값 - A)의 중앙값은 0보다 작다
- alternative='greater'
   - H0: 모집단의 중앙값은 A보다 작거나 같다.
   - H1: 모집단의 중앙값은 A보다 크다.
   - H0: 값 차이(관찰값 - A)의 중앙값은 0보다 작거나 같다
   - H1: 값 차이(관찰값 - A)의 중앙값은 0보다 크다       

중량이 100g 으로 표기된 닭가슴살 제품이 100g이라고 할 수 있는가?
- 동일한 회사 제품을 임의로 9개 표본 추출하였음
- 통계적 유의수준은 0.05로 사용함
- H0: 닭 가슴살 중량의 중앙값은 100g이다
- H1: 닭 가슴살 중량의 중앙값은 100g이 아니다

In [ ]:
# [1] 데이터 가져오기
import pandas as pd
df = pd.read_csv('data_02/chicken_breast.csv')
print(df)

       weight
0  107.704526
1   96.505220
2  109.417631
3  109.696405
4   90.783694
5  107.871426
6  106.447983
7  104.484894
8   95.778403


In [ ]:
# [2] 중앙값 구하기
median = df.weight.median()  # df['weight']
print(f'median={median:.4f}')

median=106.4480


In [ ]:
# [3] 윌콕슨 검정 실행
from scipy.stats import wilcoxon
statistic, pvalue = wilcoxon(df.weight - 100, alternative='two-sided')
print(f'statistic={statistic:.4f}, p-value={pvalue:.4f}')

statistic=10.0000, p-value=0.1641


In [ ]:
# [4] 결과
# p-value > 0.05, 귀무가설을 채택
# 닭 가슴살 중량의 중앙값은 100g이다

alternative='less'
- H0: 닭가슴살 중량의 중앙값은 100 보다 크거나 같다.
- H1: 닭가슴살 중량의 중앙값은 100 보다 작다.

In [ ]:
from scipy.stats import wilcoxon
statistic, pvalue = wilcoxon(df.weight - 100, alternative='less')
print(f'statistic={statistic:.4f}, p-value={pvalue:.4f}')
# p-value > 0.05, 귀무가설을 채택
# 닭가슴살 중량의 중앙값은 100 보다 크거나 같다.

statistic=35.0000, p-value=0.9355


alternative='greater'
- H0: 닭가슴살 중량의 중앙값은 100 보다 작거나 같다.
- H1: 닭가슴살 중량의 중앙값은 100 보다 크다.

In [ ]:
from scipy.stats import wilcoxon
statistic, pvalue = wilcoxon(df.weight - 100, alternative='greater')
print(f'statistic={statistic:.4f}, p-value={pvalue:.4f}')
# p-value > 0.05, 귀무가설을 채택
# 닭가슴살 중량의 중앙값은 100 보다 작거나 같다.

statistic=35.0000, p-value=0.0820


#### 3-2.대응표본(Paired)
- alternative='two-sided'
    - H0: 값 차이(처리후 - 처리전)의 중앙값이 0이다(=값의 차이가 없다)
    - H1: 값 차이(처리후 - 처리전)의 중앙값이 0이 아니다(=값의 차이가 있다)
- alternative='less'
    - H0: 값 차이(처리후 - 처리전)의 중앙값이 0보다 크거나 같다
    - H1: 값 차이(처리후 - 처리전)의 중앙값이 0보다 작다
- alternative='greater'
    - H0: 값 차이(처리후 - 처리전)의 중앙값이 0보다 작거나 같다
    - H1: 값 차이(처리후 - 처리전)의 중앙값이 0보다 크다 

다이어트 후, 체중이 줄었다고 할 수 있는가? 몸무게 : 비율척도
- 동일한 회사 제품을 임의로 8개 표본 추출하였음
- 통계적 유의수준은 0.05로 사용함


alternative='less'
- H0: 다이어트 후 - 다이어트 전 몸무게 중앙값은 0보다 크거나 같다 
- H1: 다이어트 후 - 다이어트 전 몸무게 중앙값은 0보다 작다.

In [ ]:
# [1] 파일 불러오기
import pandas as pd
df = pd.read_csv('data_02/diet_result.csv')
print(df)

      before      after
0  60.038972  56.153355
1  58.566940  55.678046
2  62.200320  63.010597
3  68.522263  64.413353
4  54.802465  52.110032
5  55.437637  51.330742
6  64.700073  62.047727
7  65.030917  61.157653


In [ ]:
# scipy.stats.wilcoxon(x, y=None, zero_method='wilcox', correction=False, alternative='two-sided', mode='auto')
# The Wilcoxon signed-rank test tests the null hypothesis that two related paired samples come from the same distribution. 
# In particular, it tests whether the distribution of the differences x - y is symmetric about zero.
# It is a non-parametric version of the paired T-test.

In [ ]:
# [2] wilcoxon signed rank sum - (paired t-test의 비모수)
from scipy.stats import wilcoxon
statistic, pvalue = wilcoxon(df.after, df.before, 
                             alternative='less')
print(f'statistic={statistic:.4f}, p-value={pvalue:.4f}')

statistic=1.0000, p-value=0.0078


In [ ]:
# [3] 결과
# p-value < 0.05, 귀무가설을 기각
# 다이어트 후 - 다이어트 전 몸무게 중앙값은 0보다 작다.
# 다이어트 후 몸무게가 줄었다!

alternative='two-sided'
- H0: 다이어트 후 - 다이어트 전 몸무게 중앙값은 0과 같다 
- H1: 다이어트 후 - 다이어트 전 몸무게 중앙값은 0과 다르다

In [ ]:
from scipy.stats import wilcoxon
statistic, pvalue = wilcoxon(df.after, df.before, 
                             alternative='two-sided')
print(f'statistic={statistic:.4f}, p-value={pvalue:.4f}')
# p < 0.05, 귀무가설 기각
# 다이어트 후 - 다이어트 전 몸무게 중앙값은 0과 다르다

statistic=1.0000, p-value=0.0156


alternative='greater'
- H0: 다이어트 후 - 다이어트 전 몸무게 중앙값은 0보다 작거나 같다 
- H1: 다이어트 후 - 다이어트 전 몸무게 중앙값은 0보다 크다

In [ ]:
from scipy.stats import wilcoxon
statistic, pvalue = wilcoxon(df.after, df.before, 
                             alternative='greater')
print(f'statistic={statistic:.4f}, p-value={pvalue:.4f}')
# p-value > 0.05, 귀무가설 채택
# 다이어트 후 - 다이어트 전 몸무게 중앙값은 0보다 작거나 같다

statistic=1.0000, p-value=0.9961


###4.독립 2표본(Two Sample)
- 독립 2표본 T검정에서 정규성 가정이 만족되지 않을 때 사용하는 비모수 검정법
- **두 표본의 중위수를 비교**하는데 쓰임
- 가정
   1. 두 그룹은 독립적이다
   2. 측정값은 최소 순서형 변수이다(대소 비교가 가능해야 함)

   

- alternative='two-sided'
    - H0: gA의 중앙값과 gB의 중앙값이 같다(차이가 없다)
    - H1: gA의 중앙값과 gB의 중앙값이 다르다(차이가 있다)
- alternative='less'
    - H0: gA의 중앙값이 gB의 중앙값보다 크거나 같다
    - H1: gA의 중앙값이 gB의 중앙값보다 작다
- alternative='greater'
    - H0: gA의 중앙값이 gB의 중앙값보다 작거나 같다
    - H1: gA의 중앙값이 gB의 중앙값보다 크다

#### 4-1.Wilcoxon Rank Sum Test

A사 닭가슴살 제품의 중량과 B사 닭가슴살 제품의 중량이 차이가 있는지 확인
- A사 닭가슴살은 40개의 표본, B사 닭가슴살은 20개의 표본이 있으며
- 각각 독립적이고 임의로 추출했다
- 통계적 유의수준은 0.05 사용


alternative='two-sided'
- H0: A사 닭가슴살 중량과 B사 닭가슴살 중량의 차이가 없다
- H1: A사 닭가슴살 중량과 B사 닭가슴살 중량의 차이가 있다

In [ ]:
# [1] 파일 불러오기
import pandas as pd
df = pd.read_csv('data_02/chicken_weight_AB.csv')
print(df.head())

  company      weight
0       A   97.604131
1       A   95.963639
2       A  101.239447
3       A   99.734285
4       A   99.130079


In [ ]:
# [2] 그룹 나누기
gA = df.loc[df['company']=='A', 'weight']
gB = df.loc[df['company']=='B', 'weight']
print(len(gA), len(gB))

40 20


In [ ]:
# 정규성 검정
from scipy.stats import shapiro
_, pvalueA = shapiro(gA)  # 정규성 만족
_, pvalueB = shapiro(gB)  # 정규성 만족하지 않음
print(f'pvalueA={pvalueA:.4f}, pvalueB={pvalueB:.4f}')

pvalueA=0.5676, pvalueB=0.0006


scipy.stats.ranksums(x, y, alternative='two-sided')
- Compute the Wilcoxon rank-sum statistic for two samples.
- The Wilcoxon rank-sum test tests the null hypothesis that two sets of measurements are drawn from the same distribution. 
- The alternative hypothesis is that values in one sample are more likely to be larger than the values in the other sample.

- This test should be used to compare two samples from continuous distributions. 
- It does not handle ties between measurements in x and y. For tie-handling and an optional continuity correction see scipy.stats.mannwhitneyu.

In [ ]:
# [3] wilcoxon rank sum test (two sample)
from scipy.stats import ranksums
statistic, pvalue = ranksums(gA, gB, alternative='two-sided')
print(f'statistic={statistic:.4f}, p-value={pvalue:.4f}')

statistic=-4.8612, p-value=0.0000


In [ ]:
# [4] 결과
# p-value < 0.05, 귀무가설 기각
# A사 닭가슴살 중량과 B사 닭가슴살 중량의 차이가 있다
# statistic=-4.8612 // gA가 더 작음

alternative='less'
- H0: A사 닭가슴살 중량이 B사 닭가슴살 중량보다 크거나 같다
- H1: A사 닭가슴살 중량이 B사 닭가슴살 중량보다 작다

In [ ]:
# wilcoxon rank sum test (two sample)
statistic, pvalue = ranksums(gA, gB, alternative='less')
print(f'statistic={statistic:.4f}, p-value={pvalue:.4f}')
# p-value < 0.05, 귀무가설 기각 
# A사 닭가슴살 중량이 B사 닭가슴살 중량보다 작다

statistic=-4.8612, p-value=0.0000


alternative='greater'
- H0: A사 닭가슴살 중량이 B사 닭가슴살 중량보다 작거나 같다
- H1: A사 닭가슴살 중량이 B사 닭가슴살 중량보다 크다

In [ ]:
# wilcoxon rank sum test (two sample)
from scipy.stats import ranksums
statistic, pvalue = ranksums(gA, gB, alternative='greater')
print(f'statistic={statistic:.4f}, p-value={pvalue:.4f}')
# p-value > 0.05, 귀무가설 채택 
# A사 닭가슴살 중량이 B사 닭가슴살 중량보다 작거나 같다

statistic=-4.8612, p-value=1.0000


#### 4-2.Mann-Whitney U test

alternative='two-sided'
- H0: A사 닭가슴살 중량과 B사 닭가슴살 중량의 차이가 없다
- H1: A사 닭가슴살 중량과 B사 닭가슴살 중량의 차이가 있다

scipy.stats.mannwhitneyu(x, y, use_continuity=True, alternative='two-sided', axis=0, method='auto')
- Perform the Mann-Whitney U rank test on two independent samples.
- The Mann-Whitney U test is a nonparametric test of the null hypothesis that the distribution underlying sample x is the same as the distribution underlying sample y. 


In [ ]:
# [1] Mann-Whitney U test (two sample)
from scipy.stats import mannwhitneyu
statistic, pvalue = mannwhitneyu(gA, gB, alternative='two-sided')
print(f'statistic={statistic:.4f}, p-value={pvalue:.4f}')

statistic=90.0000, p-value=0.0000


In [ ]:
# [2] 결과
# pvalue < 0.05, 귀무가설을 기각
# A사 닭가슴살 중량과 B사 닭가슴살 중량의 차이가 있다

alternative='less'
- H0: A사 닭가슴살 중량이 B사 닭가슴살 중량보다 크거나 같다
- H1: A사 닭가슴살 중량이 B사 닭가슴살 중량보다 작다

In [ ]:
from scipy.stats import mannwhitneyu
statistic, pvalue = mannwhitneyu(gA, gB, alternative='less')
print(f'statistic={statistic:.4f}, p-value={pvalue:.4f}')
# pvalue < 0.05, 귀무가설을 기각
# A사 닭가슴살 중량이 B사 닭가슴살 중량보다 작다

statistic=90.0000, p-value=0.0000


alternative='greater'
- H0: A사 닭가슴살 중량이 B사 닭가슴살 중량보다 작거나 같다
- H1: A사 닭가슴살 중량이 B사 닭가슴살 중량보다 크다

In [ ]:
from scipy.stats import mannwhitneyu
statistic, pvalue = mannwhitneyu(gA, gB, alternative='greater')
print(f'statistic={statistic:.4f}, p-value={pvalue:.4f}')
# p-value > 0.05, 귀무가설 채택 
# A사 닭가슴살 중량이 B사 닭가슴살 중량보다 작거나 같다

statistic=90.0000, p-value=1.0000


### 5.Kruskal-Wallis H-test
- 분산 분석(ANOVA)에서 정규성 가정이 만족되지 않을 때 사용하는 비모수 검정법
- ANOVA와는 달리 중앙값에 관한 결과를 얻을 수 있음
- 가정
  1. 표본은 독립적이다.
  2. 측정값은 최소 순서형 변수이다. (대소 비교가 가능해야 합니다.)
- 가설
  - H0: 모든 그룹의 중앙값은 서로 같다
  - H1: 모든 그룹의 중앙값이 전부 같은 것은 아니다

scipy.stats.kruskal(*args, nan_policy='propagate')[source]
- The Kruskal-Wallis H-test tests the null hypothesis that the population median of all of the groups are equal. 
- It is a non-parametric version of ANOVA. The test works on 2 or more independent samples, which may have different sizes. 
- Note that rejecting the null hypothesis does not indicate which of the groups differs. 
- Post hoc comparisons between groups are required to determine which groups are different.

####5-1.K개 Sample
- A, B, C 고등학교 학생들의 하루 공부 시간을 조사했을 때,고등학교 간에 공부 시간이 차이가 있는지 확인
- 통계적 유의수준은 0.05


- H0: A,B,C 고등학교 학생들의 하루 공부시간에 차이가 없다
- H1: A,B,C 고등학교 학생들의 하루 공부시간에 차이가 있다

In [ ]:
# [1] 파일 불러오기
import pandas as pd
df = pd.read_csv('data_02/high_school.csv')
df.head(2)

,group,time
0,A,12.525909
1,A,9.347533


In [ ]:
# [2] 그룹 나누기
gA = df.loc[df.group =='A', 'time']
gB = df.loc[df.group =='B', 'time']
gC = df.loc[df.group =='C', 'time']
print(len(gA), len(gB), len(gC))

20 30 15


In [ ]:
from scipy.stats import shapiro
_, pvalueA = shapiro(gA)  # 정규성 만족
_, pvalueB = shapiro(gB)  # 정규성 만족
_, pvalueC = shapiro(gC)  # 정규성 만족
print(f'A={pvalueA:.4f}, B={pvalueB:.4f}, C={pvalueC:.4f}')

A=0.4412, B=0.6307, C=0.2647


In [ ]:
# [3] Kruskal-Wallis Test
from scipy.stats import kruskal
statistic, pvalue = kruskal(gA, gB, gC)
print(f'statistic={statistic:.4f}, pvalue={pvalue:.4f}')

statistic=34.1434, pvalue=0.0000


In [ ]:
# [4] 결과
# p-value < 0.05, 귀무가설을 기각
# A,B,C 고등학교 학생들의 하루 공부시간에 차이가 있다

In [ ]:
# [5] ANOVA Test
from scipy.stats import f_oneway
statistic, pvalue = f_oneway(gA, gB, gC)
print(f'statistic={statistic:.4f}, pvalue={pvalue:.4f}')

statistic=1394.1870, pvalue=0.0000


In [ ]:
# 등분산성 검정은??  (강의 내용에 없지만 확인!)
from scipy.stats import bartlett
bartlett(gA, gB, gC)

BartlettResult(statistic=0.978756985013508, pvalue=0.6130072644235558)

In [ ]:
# 만일 정규성은 만족하는데, 등분산성을 만족하지 않는다면?
# welch's ANOVA 사용

#### 5-2.분류 모델에서 활용
- 1개 연속형 변수, 1개 범주형 변수 (4개 범주)
- 범주별로 연속형 변수의 중앙값 차이가 있는지 검정한다

In [ ]:
# [1] 데이터 읽어오기 (4회기출)
import pandas as pd

data = pd.read_csv('https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/train_04.csv')
data.head(2)

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,4
1,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,2


In [ ]:
# [2] 그룹 나누기 - 종속변수 Age, 독립변수 Segmentation
gA = data.loc[data.Segmentation==1, 'Age']
gB = data.loc[data.Segmentation==2, 'Age']
gC = data.loc[data.Segmentation==3, 'Age']
gD = data.loc[data.Segmentation==4, 'Age']
print(gA.shape, gB.shape, gC.shape, gD.shape)

(1616,) (1572,) (1720,) (1757,)


In [ ]:
# [3] 정규성 검정
from scipy.stats import shapiro
_, pvalueA = shapiro(gA)  # 정규성 만족하지 않음 (gA ~ gD)
_, pvalueB = shapiro(gB)
_, pvalueC = shapiro(gC)
_, pvalueD = shapiro(gD)
print(f'A={pvalueA:.4f}, B={pvalueB:.4f}, C={pvalueC:.4f}, D={pvalueD:.4f}')

A=0.0000, B=0.0000, C=0.0000, D=0.0000


In [ ]:
# [4] 'Segmentation' 그룹별 'Age'의 중앙값 구하기
data.groupby('Segmentation')['Age'].median()

Segmentation
1    40.0
2    46.0
3    49.0
4    29.0
Name: Age, dtype: float64

In [ ]:
# [5] 크루스칼-왈리스 검정 수행
# 귀무가설 : Segmentation별 Age의 차이가 없다
# 대립가설 : Segmentation별 Age의 차이가 있다
from scipy.stats import kruskal
statistic, pvalue = kruskal(gA, gB, gC, gD)
print(f'statistic={statistic:.4f}, p-value={pvalue:.4f}')

statistic=1448.6140, p-value=0.0000


In [ ]:
# [6] 결론
# p-value < 0.05, 귀무가설을 기각
# Segmentation별 Age의 차이가 있다